## INGESTING RAW DATA FROM AWS S3 BUCKET

In [0]:
from pyspark.sql.functions import to_timestamp, col, regexp_replace, date_format
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, BooleanType

In [0]:
%run "/Workspace/UPI Transactions/Databricks_Notebooks/ETL_Files/01_Config"

In [0]:
df_json = spark.readStream.format('cloudFiles')\
            .option('cloudFiles.format', 'json')\
            .option('cloudFiles.schemaEvolutionMode', 'addNewColumns')\
            .option('multiLine', True)\
            .option('cloudFiles.inferColumnTypes', 'true')\
            .option('cloudFiles.schemaLocation', f"{s3_bucket}/{bronze_folder}/{bronze_schema}/")\
            .load(f"{s3_bucket}/{raw_folder}/*.json")

In [0]:
df_final = df_json.withColumn('timestamp',
                              to_timestamp(regexp_replace('timestamp', 'Z$', ''), "yyyy-MM-dd'T'HH:mm:ss.SSSSSSXXX"))\
            .withColumn('timestamp', date_format(col("timestamp"), 'yyyy-MM-dd HH:mm:ss'))

In [0]:
df_final.writeStream.format('json')\
    .option('mergeSchema', 'true')\
    .option('checkpointLocation', f"{s3_bucket}/{bronze_folder}/{checkpoint_location}/")\
    .outputMode('append')\
    .trigger(availableNow=True)\
    .start(f"{s3_bucket}/{bronze_folder}/json_files/")